In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
from scipy import stats
from matplotlib import font_manager, rc
#by_60211642
data = pd.read_csv("./코로나바이러스감염증-19_확진환자_발생현황_221126.csv", header = 0)
data.tail(10)

,일자,계(명),국내발생(명),해외유입(명),사망(명),Unnamed: 5,열2,열3,열4,열5,Unnamed: 10,Unnamed: 11
1033,2022.11.18,"49,399","49,333",66,63,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1034,2022.11.19,"50,568","50,518",50,65,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1035,2022.11.20,"45,998","45,940",58,41,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1036,2022.11.21,"23,077","23,016",61,35,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1037,2022.11.22,"72,860","72,779",81,45,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1038,2022.11.23,"70,306","70,237",69,53,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1039,2022.11.24,"59,082","59,027",55,59,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1040,2022.11.25,"53,698","53,659",39,55,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1041,2022.11.26,"52,788","52,726",62,52,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1042,누적(명),"26,890,488","26,819,311","71,177","30,330",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# 맥 개발환경 한국어 설정을 위함.
import matplotlib as mpl
#by_60211642
print(mpl.matplotlib_fname())

/Users/jieunsong/opt/anaconda3/lib/python3.9/site-packages/matplotlib/mpl-data/matplotlibrc


In [7]:
# 맥 개발환경 한국어 설정을 위함.
font_path="./NanumGothicBold.ttf"
font_name=font_manager.FontProperties(fname=font_path).get_name()
rc("font",family=font_name)
#by_60211642

In [8]:
df_covid = data.loc[:,['일자','계(명)','사망(명)']][:1042]
df_accumulate = data.loc[:,['일자','계(명)','사망(명)']][1042:1043] 
df_covid
#by_60211642

,일자,계(명),사망(명)
0,2020.1.20,1,-
1,2020.1.21,0,-
2,2020.1.22,0,-
3,2020.1.23,0,-
4,2020.1.24,1,-
...,...,...,...
1037,2022.11.22,"72,860",45
1038,2022.11.23,"70,306",53
1039,2022.11.24,"59,082",59
1040,2022.11.25,"53,698",55


In [9]:
df_accumulate
#by_60211642

,일자,계(명),사망(명)
1042,누적(명),"26,890,488","30,330"


In [10]:
df_covid.isna().sum()
#by_60211642
# "-"이 NaN으로 인식이 안되는 상황 0 으로 고쳐져야 해결이 됨을 알 수 있음.

일자       0
계(명)     0
사망(명)    0
dtype: int64

In [11]:
df_covid = df_covid.replace("-", 0).set_index('일자')
df_covid
#by_60211642

,계(명),사망(명)
일자,,
2020.1.20,1,0
2020.1.21,0,0
2020.1.22,0,0
2020.1.23,0,0
2020.1.24,1,0
...,...,...
2022.11.22,"72,860",45
2022.11.23,"70,306",53
2022.11.24,"59,082",59


In [12]:
df_covid.isna().sum()
#by_60211642

계(명)     0
사망(명)    0
dtype: int64

In [13]:
# 해당 날짜까지 누적 확진자 수 필요.
#by_60211642
df_covid["누적 확진자 수"] = df_covid["계(명)"]
def cumulative_number(i):
    if i == 0:
        df_covid["누적 확진자 수"][0] = int(df_covid["계(명)"][0])
    else:
        if type(df_covid["계(명)"][i]) != int :
            p = df_covid["계(명)"][i].split(",")
            str_confirmed = ""
            num_confirmed = 0
            for j in range(len(p)):
                str_confirmed+=p[j]
            df_covid["계(명)"][i] = int(str_confirmed)
            
        df_covid["누적 확진자 수"][i] = int(df_covid["누적 확진자 수"][i-1]) + df_covid["계(명)"][i]

# df_covid["누적 확진자 수"]
for i in range(len(df_covid)):
    cumulative_number(i)
    
df_covid

,계(명),사망(명),누적 확진자 수
일자,,,
2020.1.20,1,0,1
2020.1.21,0,0,1
2020.1.22,0,0,1
2020.1.23,0,0,1
2020.1.24,1,0,2
...,...,...,...
2022.11.22,72860,45,26654614
2022.11.23,70306,53,26724920
2022.11.24,59082,59,26784002


#by_60211642
# 전체 국민 수 데이터가 없는 관계로, KOSIS (통계청, 장래인구추계) 통계청 중위 추계 기준
# 5,162만 8,117명으로 한다.
rok_pop = 51628117
#감염률 p_i 감염자수 / (전체 국민수 - 전날까지의 누적 감염자수)
#분모
denominator[i] = (rok_pop - df_covid["누적 확진자 수"][i-1])
#분자
numerator[i] = df_covid["계(명)"][i]

p_infection = numerator[i]/denominator[i]

In [89]:
#by_60211642
df_covid_etc = df_covid.loc['2020.1.20':"2021.7.31"]
df_covid_delta = df_covid.loc["2021.7.31":"2022.1.24"]
df_covid_omi = df_covid.loc["2022.1.24":]
df_covid_etc

,계(명),사망(명)
일자,,
2020.1.20,1,0
2020.1.21,0,0
2020.1.22,0,0
2020.1.23,0,0
2020.1.24,1,0
...,...,...
2021.7.27,"1,363",2
2021.7.28,"1,895",4
2021.7.29,"1,673",2


In [90]:
df_covid_delta

,계(명),사망(명)
일자,,
2021.7.31,"1,538",6
2021.8.1,"1,442",3
2021.8.2,"1,218",1
2021.8.3,"1,200",5
2021.8.4,"1,725",2
...,...,...
2022.1.20,"6,599",28
2022.1.21,"6,763",21
2022.1.22,"6,999",28


In [91]:
df_covid_omi

,계(명),사망(명)
일자,,
2022.1.24,"7,510",25
2022.1.25,"8,568",23
2022.1.26,"13,002",32
2022.1.27,"14,514",34
2022.1.28,"16,089",24
...,...,...
2022.11.22,"72,860",45
2022.11.23,"70,306",53
2022.11.24,"59,082",59
